In [3]:
import pandas as pd
import numpy as np
from timedelta import Timedelta 

from lumibot.brokers import Alpaca
from lumibot.backtesting import BacktestingBroker, YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from lumibot.entities import Asset
from datetime import datetime 
from alpaca_trade_api import REST 

from src.get_data import GetUpdateData
from src.get_data import GetDataFRED
from ta import add_all_ta_features

from joblib import dump, load

In [18]:
import json

# Load secrets from secrets.json
with open('secrets.json') as f:
    secrets = json.load(f)
    
api_key_twelve = secrets['default']['twelve_data_api']
api_key_fred = secrets['default']['fred_api']

In [35]:
clf_model = load(
    "models/candidate_models/lgbm-direction-classifier-2024-05-13-00-01-11.joblib"
)
reg_model = load(
    "models/candidate_models/lgbm-percentchange-regressor-2024-05-13-00-03-32.joblib"
)


class MyStrategy(Strategy):
    def initialize(self, symbol: str = "SPY", budget: int = 1000, cash_at_risk: float = 1):
        self.symbol = symbol
        self.budget = budget
        self.sleeptime = "7D"
        self.last_trade = None
        # self.set_market("24/7")
        self.cash_at_risk = cash_at_risk
        self.df_ohlci = self.get_data()

    def get_data(self):

        ticker = self.symbol.replace("-", "/")
        df_stock = (
            GetUpdateData(api_key=api_key_twelve)
            .get_data_single_ticker(ticker)
            .sort_values("datetime", ascending=True)
        )

        _, df_tickers = GetUpdateData(
            update_existing=False, api_key=api_key_twelve
        ).get_tickers()

        df_stock = df_stock.merge(
            df_tickers[["ticker", "GICS Sector", "GICS Sub-Industry"]],
            on="ticker",
            how="left",
        )
        
        df_fred = GetDataFRED(api_key=api_key_fred).get_data()
        df_fred["datetime"] = pd.to_datetime(df_fred["datetime"])

        df_ohlci = add_all_ta_features(
            df_stock,
            open="open",
            high="high",
            low="low",
            close="close",
            volume="volume",
            fillna=True,
        )

        df_ohlci["datetime"] = pd.to_datetime(df_ohlci["datetime"])
        df_ohlci = df_ohlci.merge(df_fred, on="datetime", how="left")

        df_ohlci["year"] = df_ohlci["datetime"].dt.year
        df_ohlci["month"] = df_ohlci["datetime"].dt.month
        df_ohlci["dayofweek"] = df_ohlci["datetime"].dt.weekday

        print(df_ohlci["datetime"].min())

        return df_ohlci

    def get_date(self):
        today = self.get_datetime()
        return today.strftime("%Y-%m-%d")

    def get_signal(self):
        X = self.df_ohlci.copy()
        today = self.get_date()
        X = X.reset_index(drop=True)
        prev_index = X[X["datetime"] == today].index[0] - 1
        X = X.iloc[[prev_index]]
        X.drop("datetime", axis=1, inplace=True)

        pred_direction_proba = clf_model.predict_proba(X)[0][1]
        pred_percent_change = reg_model.predict(X)[0]

        # print(f'today_date: {today}, X Empty?: {X.empty}')

        return pred_direction_proba, pred_percent_change, X

    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price, 0)
        return cash, last_price, quantity

    def on_trading_iteration(self):

        cash, last_price, quantity = self.position_sizing()
        pred_direction_proba, pred_percent_change, X = self.get_signal()
        yesterday_close_price = X["close"].values[0]

        print(
            f"pred_direction_proba: {pred_direction_proba}, pred_percent_change: {pred_percent_change}"
        )

        if (cash > last_price) & (pred_direction_proba >= 0.51):
            # if  pred_percent_change >= 0.001: #(last_price < (yesterday_close_price * (pred_percent_change * 100))):
            if self.last_trade == "sell":
                self.sell_all()
            order = self.create_order(
                self.symbol,
                quantity,
                "buy",
                type="bracket",
                take_profit_price=last_price * 1.10,
                stop_loss_price=last_price * 0.95,
            )
            self.submit_order(order)
            self.last_trade = "buy"
            # elif pred_direction == 1 and pred_percent_change <= 0.01:
            #     self.sell_all()
            # elif pred_direction == 0 and pred_percent_change >= -0.01:
            #     self.sell_all()
        if (
            pred_direction_proba <= 0.45
        ):  # (last_price >= (yesterday_close_price * (pred_percent_change * 100))):
            if self.last_trade == "buy":
                self.sell_all()
                cash, last_price, quantity = self.position_sizing()
                if cash > last_price:
                    order = self.create_order(
                        self.symbol,
                        quantity,
                        "sell",
                        type="bracket",
                        take_profit_price=last_price * 0.90,
                        stop_loss_price=last_price * 1.05,
                    )
                    self.submit_order(order)
                    self.last_trade = "sell"


# Pick the dates that you want to start and end your backtest
# and the allocated budget
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 4, 30)
# broker = Alpaca(ALPACA_CREDS)

# Run the backtest
trader = Trader(backtest=True)
data_source = YahooDataBacktesting(
    datetime_start=start_date,
    datetime_end=end_date,
)
broker = BacktestingBroker(data_source)
strat = MyStrategy(
    broker=broker,
)
trader.add_strategy(strat)
trader.run_all()

Getting DGS10...
DGS10 [min-max] date: [1962-01-02 00:00:00-2024-05-09 00:00:00]
Getting FEDFUNDS...
FEDFUNDS [min-max] date: [1954-07-01 00:00:00-2024-04-01 00:00:00]
Getting GDP...
GDP [min-max] date: [1946-01-01 00:00:00-2024-01-01 00:00:00]
Getting UNRATE...
UNRATE [min-max] date: [1948-01-01 00:00:00-2024-04-01 00:00:00]
Getting CP...
CP [min-max] date: [1946-01-01 00:00:00-2023-10-01 00:00:00]
Getting WM2NS...
WM2NS [min-max] date: [1980-11-03 00:00:00-2024-04-01 00:00:00]
Getting NASDAQCOM...
NASDAQCOM [min-max] date: [1971-02-05 00:00:00-2024-05-09 00:00:00]
Getting UMCSENT...
UMCSENT [min-max] date: [1952-11-01 00:00:00-2024-03-01 00:00:00]
Getting HSN1F...
HSN1F [min-max] date: [1963-01-01 00:00:00-2024-03-01 00:00:00]
Getting MSPNHSUS...
MSPNHSUS [min-max] date: [1963-01-01 00:00:00-2024-03-01 00:00:00]
2004-07-01 00:00:00
Progress ||   0.09%  [Elapsed: 0:00:07 ETA: 2:29:28] pred_direction_proba: 0.5651186028709819, pred_percent_change: -0.0008278820471694871
Progress ||   0

{'MyStrategy': {'cagr': 1.0765779949235195,
  'volatility': 0.20716219569001257,
  'sharpe': 4.943942558396673,
  'max_drawdown': {'drawdown': 0.08307594355630739,
   'date': Timestamp('2021-02-23 09:30:00-0500', tz='America/New_York')},
  'romad': 12.958961991130852,
  'total_return': 22.31166444830319}}